In [ ]:
pip install requests beautifulsoup4 pandas

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from pathlib import Path
import csv


# url for UCSC Baskin courses
bsoe_base_url = "https://courses.engineering.ucsc.edu/"

# fetch the base page and parse the department links
base = requests.get(bsoe_base_url)
soup_base = BeautifulSoup(base.content, 'html.parser')
department_menu = soup_base.find("ul", id="main-menu")

# gather department links
department_links = []
for department in department_menu:
    if department != "\n":
        if link := department.find("a"):
            department_links.append(
                (bsoe_base_url + link["href"][1:], link.text))

department_links.pop()
print(department_links)

# initialize a list to hold course data
course_data = []

# iterate through department links and extract course information
for link, name in department_links:
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find("table")
    sections = table.find_all("li")

    for section in sections:
        link = section.find("a")
        section_num = link.text
        class_url = link["href"]
        class_res = re.search(
            "\/[a-z]*\/([A-Z0-9]*)\/([A-Za-z0-9]*)\/", class_url)

        if class_res:
            class_name = class_res.group(1)  # extract class name
            quarter = class_res.group(2)  # extract quarter
            full_prof = section.contents[3].strip() if len(section.contents) > 3 else ''

            # extract professor name, checking if it exists
            prof_res = re.search(r"([ a-zA-ZÀ-ž-.]*) \(|(Staff)", full_prof)
            professor_name = prof_res.group(1) or prof_res.group(2)

            # skip the course if no professor is found
            # case where the class is not taught at all that school year
            if not professor_name or professor_name.strip().lower() == "staff":
                continue

            # prepare the data for CSV
            course_data.append({
                "department": name,
                "quarter": quarter,
                "class_name": class_name,
                "professor_name": professor_name,
                "section": section_num,
            })
            print(course_data[-1])  # print the last added data

# write data to a CSV file
csv_file = 'ucsc_baskin_courses_2024.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    fieldnames = ["department", "quarter", "class_name", "professor_name", "section"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(course_data)

print(f"\nData successfully saved to {csv_file}")


[('https://courses.engineering.ucsc.edu/courses/am', 'Applied Mathematics'), ('https://courses.engineering.ucsc.edu/courses/bme', 'Biomolecular Engineering'), ('https://courses.engineering.ucsc.edu/courses/cmpm', 'Computational Media'), ('https://courses.engineering.ucsc.edu/courses/cse', 'Computer Science and Engineering'), ('https://courses.engineering.ucsc.edu/courses/ece', 'Electrical and Computer Engineering'), ('https://courses.engineering.ucsc.edu/courses/game', 'Games and Playable Media'), ('https://courses.engineering.ucsc.edu/courses/hci', 'Human Computer Interaction'), ('https://courses.engineering.ucsc.edu/courses/nlp', 'Natural Language Processing'), ('https://courses.engineering.ucsc.edu/courses/game', 'Serious Games'), ('https://courses.engineering.ucsc.edu/courses/stat', 'Statistics'), ('https://courses.engineering.ucsc.edu/courses/tim', 'Technology & Information Management')]
{'department': 'Applied Mathematics', 'quarter': 'Fall24', 'class_name': 'AM3', 'professor_nam

In [ ]:
import random

# based on:
# https://catalog.ucsc.edu/en/current/general-catalog/academic-units/baskin-engineering/computer-science-and-engineering/computer-science-bs/

# all courses that must be taken and have no option choices
must_take = {

    # lower divs
    "CSE 12": {"credits": 7, "prerequisites": [["CSE 5J", "CSE 20", "CSE 30", "BME 160"]]},
    "CSE 16": {"credits": 5, "prerequisites": [["MATH 19A", "MATH 19B", "MATH 11B", "AM 11B", "AM 15B", "ECON 11B"]]},
    "CSE 20": {"credits": 5, "prerequisites": []},
    "CSE 30": {"credits": 5, "prerequisites": [["CSE 20", "BME 160", "MATH 3", "MATH 11A", "MATH 19A", "AM 3", "AM 11A"]]},
    "CSE 40": {"credits": 5, "prerequisites": [["MATH 19B", "MATH 20B"], ["CSE 30"] ]},
    "CSE 13S": {"credits": 7, "prerequisites": [["CSE 12", "BME 160"]]},
    "ECE 30": {"credits": 5, "prerequisites": [["MATH 19B"]]},

    # upper divs
    "CSE 101": {"credits": 5, "prerequisites": [["CSE 12", "BME 160"], ["CSE 13E", "ECE 13", "CSE 13S"],
        ["CSE 16"], ["CSE 30"], ["MATH 11B", "MATH 19B", "MATH 20B", "AM 11B", "ECON 11B"]]},
    "CSE 101M": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 120": {"credits": 5, "prerequisites": [["CSE 12"], ["CSE 13S", "CSE 13E", "ECE 13"]]},
    "CSE 130": {"credits": 5, "prerequisites": [["CSE 12"],["CSE 101"]]},
}

# must pick 1 calc 1 class
calc_1 = {
    "MATH 19A": {"credits": 5, "prerequisites": []},
    "MATH 20A": {"credits": 5, "prerequisites": []}
}

# must pick 1 calc 2 class
calc_2 = {
    "MATH 19B": {"credits": 5, "prerequisites": [["MATH 19A", "MATH 20A"]]},
    "MATH 20B": {"credits": 5, "prerequisites": [["MATH 20A"]]}
}

# must pick 1 calc 3 class
calc_3 = {
    "AM 30": {"credits": 5, "prerequisites": [["AM 10", "MATH 21"], ["MATH 19B", "MATH 20B"]]},
    "MATH 21": {"credits": 5, "prerequisites": [["MATH 19A"]]}
}

# must pick 1 one linear alg class
linear_alg = {
    "AM 10": {"credits": 5, "prerequisites": []},
    "MATH 21": {"credits": 5, "prerequisites": [["MATH 19A"]]}
}

# must pick 1 stats class
stats = {
    "STAT 131": {"credits": 5, "prerequisites": [["AM 11B", "ECON 11B", "MATH 11B", "MATH 19B", "MATH 20B"]]},
    "CSE 107": {"credits": 5, "prerequisites": [["CSE 16"], ["AM 30", "MATH 22", "MATH 23A"]]}
}

# must pick 1 theory class
theory = {
    "CSE 102": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 103": {"credits": 5, "prerequisites": [["CSE 101"]]}
}

# must pick 1 programming lang class
theory = {
    "CSE 112": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 114A": {"credits": 5, "prerequisites": [["CSE 101"]]}
}

# must take 3 elective fillers here, plus a capstone that counts as an elective (listed after)
electives = {
    "elective 1": {"credits": 5, "prerequisites": []},
    "elective 2": {"credits": 5, "prerequisites": []},
    "elective 3": {"credits": 5, "prerequisites": []},
}

# must pick 1 capstone that counts as an elective
capstone = {
    "CSE 110B": {"credits": 5, "prerequisites": [["CSE 110A"]]},
    "CSE 115C": {"credits": 5, "prerequisites": [["CSE 115B"]]},
    "CSE 115D": {"credits": 5, "prerequisites": [["CSE 115A"]]},
    "CSE 121": {"credits": 7, "prerequisites": [["CSE 12", "CSE 100", "CSE 100L"],
            ["CSE 13E", "CSE 13S", "ECE 13", "CSE 15", "CSE 15L"],
            ["ECE 101", "ECE 101L", "PHYS 5C", "PHYS 5N"]
        ]
    },
    "CSE 134": {"credits": 5, "prerequisites": [["CSE 120", "CSE 130"]]},
    "CSE 138": {"credits": 5, "prerequisites": [["CSE 130", "CSE 131"]]},
    "CSE 140": {"credits": 5, "prerequisites": [["CSE 101", ["CSE 40", "STAT 132"]]]},
    "CSE 143": {
        "credits": 5,
        "prerequisites": [
            ["CSE 101"],
            ["CSE 107", "STAT 131"],
            ["CSE 40"]
        ]
    },
    "CSE 144": {
        "credits": 5,
        "prerequisites": [
            ["CSE 40", "STAT 132"],
            ["CSE 101"]
        ]
    },
    "CSE 145": {
        "credits": 5,
        "prerequisites": [
            ["CSE 15", "CSE 15L"],
            ["CSE 30"],
            ["CSE 13S"],
            ["AM 30", "MATH 22", "MATH 23A"],
            ["STAT 5", "CSE 107", "STAT 131"],
            ["AM 10", "MATH 21"],
            ["CSE 16", "ECON 113"]
        ]
    },
    "CSE 156": {
        "credits": 5,
        "prerequisites": [["CSE 150", "CSE 101"]],
        "concurrent": "CSE 156L"
    },
    "CSE 156L": {
        "credits": 2,
        "prerequisites": [["CSE 150", "CSE 101"]],
        "concurrent": "CSE 156"
    },
    "CSE 157": {
        "credits": 7,
        "prerequisites": [["CSE 121", "CSE 150"]]
    },
    "CSE 160": {
        "credits": 7,
        "prerequisites": [["CSE 101", ["MATH 21", "AM 10"]]]
    },
    "CSE 161": {
        "credits": 5,
        "prerequisites": [["CSE 160", "equivalent"]],
        "concurrent": "CSE 161L"
    },
    "CSE 161L": {
        "credits": 2,
        "concurrent": "CSE 161"
    },
    "CSE 162": {
        "credits": 5,
        "prerequisites": [["CSE 160", "equivalent"]],
        "concurrent": "CSE 162L"
    },
    "CSE 162L": {
        "credits": 2,
        "concurrent": "CSE 162"
    },
    "CSE 163": {
        "credits": 5,
        "prerequisites": [["CSE 101"]]
    },
    "CSE 168": {
        "credits": 7,
        "prerequisites": [["CSE 160"]]
    },
    "CSE 181": {
        "credits": 5,
        "prerequisites": [["CSE 180", "CSE 130"]]
    },
    "CSE 183": {
        "credits": 5,
        "prerequisites": [
            ["CSE 15", "CSE 15L"],
            ["CMPM 35"],
            ["CSE 101"]
        ]
    },
    "CSE 184": {
        "credits": 5,
        "prerequisites": [["CSE 101"]]
    },
    "CSE 187": {
        "credits": 5,
        "prerequisites": [["CSE 186"]]
    },
    "CMPM 172": {
        "credits": 7,
        "prerequisites": [["CMPM 171"]]
    },
}



In [ ]:
import random

# based on:
# https://catalog.ucsc.edu/en/current/general-catalog/academic-units/baskin-engineering/computer-science-and-engineering/computer-science-bs/

# all courses that must be taken and have no option choices
must_take = {

    # lower divs
    "CSE 12": {"credits": 7, "prerequisites": [["CSE 5J", "CSE 20", "CSE 30", "BME 160"]]},
    "CSE 16": {"credits": 5, "prerequisites": [["MATH 19A", "MATH 19B", "MATH 11B", "AM 11B", "AM 15B", "ECON 11B"]]},
    "CSE 20": {"credits": 5, "prerequisites": []},
    "CSE 30": {"credits": 5, "prerequisites": [["CSE 20", "BME 160", "MATH 3", "MATH 11A", "MATH 19A", "AM 3", "AM 11A"]]},
    "CSE 40": {"credits": 5, "prerequisites": [["MATH 19B", "MATH 20B"], ["CSE 30"] ]},
    "CSE 13S": {"credits": 7, "prerequisites": [["CSE 12", "BME 160"]]},
    "ECE 30": {"credits": 5, "prerequisites": [["MATH 19B"]]},

    # upper divs
    "CSE 101": {"credits": 5, "prerequisites": [["CSE 12", "BME 160"], ["CSE 13E", "ECE 13", "CSE 13S"],
        ["CSE 16"], ["CSE 30"], ["MATH 11B", "MATH 19B", "MATH 20B", "AM 11B", "ECON 11B"]]},
    "CSE 101M": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 120": {"credits": 5, "prerequisites": [["CSE 12"], ["CSE 13S", "CSE 13E", "ECE 13"]]},
    "CSE 130": {"credits": 5, "prerequisites": [["CSE 12"],["CSE 101"]]},
}

# must pick 1 calc 1 class
calc_1 = {
    "MATH 19A": {"credits": 5, "prerequisites": []},
    "MATH 20A": {"credits": 5, "prerequisites": []}
}

# must pick 1 calc 2 class
calc_2 = {
    "MATH 19B": {"credits": 5, "prerequisites": [["MATH 19A", "MATH 20A"]]},
    "MATH 20B": {"credits": 5, "prerequisites": [["MATH 20A"]]}
}

# must pick 1 calc 3 class
calc_3 = {
    "AM 30": {"credits": 5, "prerequisites": [["AM 10", "MATH 21"], ["MATH 19B", "MATH 20B"]]},
    "MATH 23A": {"credits": 5, "prerequisites": [["MATH 19A"]]}
}

# must pick 1 one linear alg class
linear_alg = {
    "AM 10": {"credits": 5, "prerequisites": []},
    "MATH 21": {"credits": 5, "prerequisites": [["MATH 19A"]]}
}

# must pick 1 stats class
stats = {
    "STAT 131": {"credits": 5, "prerequisites": [["AM 11B", "ECON 11B", "MATH 11B", "MATH 19B", "MATH 20B"]]},
    "CSE 107": {"credits": 5, "prerequisites": [["CSE 16"], ["AM 30", "MATH 22", "MATH 23A"]]}
}

# must pick 1 theory class
theory = {
    "CSE 102": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 103": {"credits": 5, "prerequisites": [["CSE 101"]]}
}

# must pick 1 programming lang class
theory = {
    "CSE 112": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 114A": {"credits": 5, "prerequisites": [["CSE 101"]]}
}

# must take 3 elective fillers here, plus a capstone that counts as an elective (listed after)
electives = {
    "elective 1": {"credits": 5, "prerequisites": []},
    "elective 2": {"credits": 5, "prerequisites": []},
    "elective 3": {"credits": 5, "prerequisites": []},
}

# must pick 1 capstone that counts as an elective
capstone = {
    "CSE 110B": {"credits": 5, "prerequisites": [["CSE 110A"]]},
    "CSE 115C": {"credits": 5, "prerequisites": [["CSE 115B"]]},
    "CSE 115D": {"credits": 5, "prerequisites": [["CSE 115A"]]},
    "CSE 121": {"credits": 7, "prerequisites": [["CSE 12", "CSE 100", "CSE 100L"],
            ["CSE 13E", "CSE 13S", "ECE 13", "CSE 15", "CSE 15L"],
            ["ECE 101", "ECE 101L", "PHYS 5C", "PHYS 5N"]
        ]
    },
    "CSE 134": {"credits": 5, "prerequisites": [["CSE 120", "CSE 130"]]},
    "CSE 138": {"credits": 5, "prerequisites": [["CSE 130", "CSE 131"]]},
    "CSE 140": {"credits": 5, "prerequisites": [["CSE 101", ["CSE 40", "STAT 132"]]]},
    "CSE 143": {
        "credits": 5,
        "prerequisites": [
            ["CSE 101"],
            ["CSE 107", "STAT 131"],
            ["CSE 40"]
        ]
    },
    "CSE 144": {
        "credits": 5,
        "prerequisites": [
            ["CSE 40", "STAT 132"],
            ["CSE 101"]
        ]
    },
    "CSE 145": {
        "credits": 5,
        "prerequisites": [
            ["CSE 15", "CSE 15L"],
            ["CSE 30"],
            ["CSE 13S"],
            ["AM 30", "MATH 22", "MATH 23A"],
            ["STAT 5", "CSE 107", "STAT 131"],
            ["AM 10", "MATH 21"],
            ["CSE 16", "ECON 113"]
        ]
    },
    "CSE 156": {
        "credits": 5,
        "prerequisites": [["CSE 150", "CSE 101"]],
        "concurrent": "CSE 156L"
    },
    "CSE 156L": {
        "credits": 2,
        "prerequisites": [["CSE 150", "CSE 101"]],
        "concurrent": "CSE 156"
    },
    "CSE 157": {
        "credits": 7,
        "prerequisites": [["CSE 121", "CSE 150"]]
    },
    "CSE 160": {
        "credits": 7,
        "prerequisites": [["CSE 101", ["MATH 21", "AM 10"]]]
    },
    "CSE 161": {
        "credits": 5,
        "prerequisites": [["CSE 160", "equivalent"]],
        "concurrent": "CSE 161L"
    },
    "CSE 161L": {
        "credits": 2,
        "concurrent": "CSE 161"
    },
    "CSE 162": {
        "credits": 5,
        "prerequisites": [["CSE 160", "equivalent"]],
        "concurrent": "CSE 162L"
    },
    "CSE 162L": {
        "credits": 2,
        "concurrent": "CSE 162"
    },
    "CSE 163": {
        "credits": 5,
        "prerequisites": [["CSE 101"]]
    },
    "CSE 168": {
        "credits": 7,
        "prerequisites": [["CSE 160"]]
    },
    "CSE 181": {
        "credits": 5,
        "prerequisites": [["CSE 180", "CSE 130"]]
    },
    "CSE 183": {
        "credits": 5,
        "prerequisites": [
            ["CSE 15", "CSE 15L"],
            ["CMPM 35"],
            ["CSE 101"]
        ]
    },
    "CSE 184": {
        "credits": 5,
        "prerequisites": [["CSE 101"]]
    },
    "CSE 187": {
        "credits": 5,
        "prerequisites": [["CSE 186"]]
    },
    "CMPM 172": {
        "credits": 7,
        "prerequisites": [["CMPM 171"]]
    },
}







TypeError: unhashable type: 'list'

In [133]:
import random

def can_take_course(course, prereqs, completed_courses):
  # print(course)
  # print(prereqs)
  # print("\n")
  satisfied = 0
  for prereq in prereqs:
    if len(prereq) > 1:
      for option in prereq:
        if option in completed_courses:
          satisfied += 1
          break
    else:
      if prereq[0] in completed_courses:
        satisfied += 1
  if satisfied == len(prereqs):
    return True
  return False

def calc_courses():

    # select which calculus class to take
    calc1_course = random.choice(list(calc_1.keys()))

    if calc1_course == "MATH 19A":
      calc2_course = "MATH 19B"
    else: # taking honors calc series
      calc2_course = "MATH 20B"

    calc3_course = random.choice(list(calc_3.keys()))

    return calc1_course, calc2_course, calc3_course


def generate_schedule(courses):
    schedule = [[], [], [],
                [], [], [],
                [], [], [],
                ["ELECTIVE"], ["ELECTIVE"], ["ELECTIVE", "CAPSTONE"],]

    completed_courses = set()

    ### calc 1-3 ###

    [calc1_course, calc2_course, calc3_course] = calc_courses()

    start_calc_quarter = random.randint(0, 0)

    schedule[start_calc_quarter].append(calc1_course)
    schedule[start_calc_quarter+1].append(calc2_course)
    schedule[start_calc_quarter+2].append(calc3_course)

    print(f"calcs: {schedule}")

    quarter = 0

    ### linear alg ###

    # select which linear alg class to take
    alg_course = random.choice(list(linear_alg.keys()))

    if alg_course == "AM 10":
      start_alg_quarter = random.randint(0, 4)
    else:
      start_alg_quarter = random.randint(start_calc_quarter+1, 4)
    schedule[start_alg_quarter].append(alg_course)

    print(f"after adding linear alg: {schedule}\n")

    ### stats ###

    # select which stats class to take, and then add it to courses
    stats_course = random.choice(list(stats.keys()))

    if stats_course == "CSE 107":
      courses["CSE 107"] = {"credits": 5, "prerequisites": [["CSE 16"], ["AM 30", "MATH 22", "MATH 23A"]]}
    else:
      courses["STAT 131"] = {"credits": 5, "prerequisites": [["AM 11B", "ECON 11B", "MATH 11B", "MATH 19B", "MATH 20B"]]}

    print(f"after adding STATS TO COURSES: {courses}\n")


    ### theory ###

    # select which stats class to take, and then add it to courses
    theory_course = random.choice(list(theory.keys()))

    if theory_course == "CSE 102":
      courses["CSE 102"] = {"credits": 5, "prerequisites": [["CSE 101"]]}
    else:
      courses["CSE 103"] = {"credits": 5, "prerequisites": [["CSE 101"]]}

    print(f"after adding theory TO COURSES: {courses}\n")


    ### pl ###

    # select which pl class to take, and then add it to courses
    pl_course = random.choice(list(pl.keys()))


    if pl_course == "CSE 112":
      courses["CSE 112"] = {"credits": 5, "prerequisites": [["CSE 101"]]}
      print(courses["CSE 112"])
    else:
      courses["CSE 114A"] = {"credits": 5, "prerequisites": [["CSE 101"]]}
      print(courses["CSE 114A"])


    ### DC ###

    # select which DC class to take, and then add it to courses
    DC_course = random.choice(list(DCs.keys()))
    # print(f"DC: {DC_course}, {DCs[DC_course]}")

    courses[DC_course] = DCs[DC_course]
    print(f"\n\nDC course: {DC_course}")
    print(f"DC prereqs: {courses[DC_course]}\n\n")


    ### capstone ###

    # # select which capstone class to take, and then add it to courses
    # capstone_course = random.choice(list(capstones.keys()))
    # print(f"CAPSTONE: {capstone_course}, {capstones[capstone_course]}")

    # courses[capstone_course] = capstones[capstone_course]
    # print(courses[capstone_course])


    # if capstone_course == "CSE 112":
    #   courses["CSE 112"] = {"credits": 5, "prerequisites": [["CSE 101"]]}
    # else:
    #   courses["CSE 114A"] = {"credits": 5, "prerequisites": [["CSE 101"]]}


    while courses:

        if quarter != 0:
          print(f"here schedule: {schedule}")
          print(quarter-1)
          for c in schedule[quarter-1]:
            completed_courses.add(c)

        # current quarter's classes
        current_quarter = []

        # list of available courses that meet prerequisites
        available_courses = []

        # collect courses that can be taken this quarter
        for course in courses:
          prereq = courses[course]["prerequisites"]
          if can_take_course(course, prereq, completed_courses):
            available_courses.append(course)

        print(f"what can be taken this quarter: {available_courses}")

        # randomly select up to 2 courses from the available courses
        random.shuffle(available_courses)  # shuffle the available courses for randomness
        # alr = len(schedule[quarter])
        for course in available_courses[:2]:
          if len(schedule[quarter]) == 2:
            break
          current_quarter.append(course)
          completed_courses.add(course)
          del courses[course]
          # add the course to the schedule
          schedule[quarter].append(course)



        print(f"updated schedule: {schedule}\n")

        # stop if we can't take any more courses this quarter
        # if not current_quarter:
        #     break

        quarter += 1

    return schedule

# define the courses as given
must_take = {
    "CSE 12": {"credits": 7, "prerequisites": [["CSE 5J", "CSE 20", "CSE 30", "BME 160"]]},
    "CSE 16": {"credits": 5, "prerequisites": [["MATH 20A", "MATH 19A", "MATH 19B", "MATH 11B", "AM 11B", "AM 15B", "ECON 11B"]]},
    "CSE 20": {"credits": 5, "prerequisites": []},
    "CSE 30": {"credits": 5, "prerequisites": [["CSE 20", "BME 160", "MATH 3", "MATH 11A", "MATH 19A", "AM 3", "AM 11A"]]},
    "CSE 40": {"credits": 5, "prerequisites": [["MATH 19B", "MATH 20B"], ["CSE 30"]]},
    "CSE 13S": {"credits": 7, "prerequisites": [["CSE 12", "BME 160"]]},
    "ECE 30": {"credits": 5, "prerequisites": [["MATH 19B", "MATH 20B"]]},

    "CSE 101": {"credits": 5, "prerequisites": [["CSE 12", "BME 160"], ["CSE 13E", "ECE 13", "CSE 13S"],
        ["CSE 16"], ["CSE 30"], ["MATH 11B", "MATH 19B", "MATH 20B", "AM 11B", "ECON 11B"]]},
    "CSE 101M": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 120": {"credits": 5, "prerequisites": [["CSE 12"], ["CSE 13S", "CSE 13E", "ECE 13"]]},
    "CSE 130": {"credits": 5, "prerequisites": [["CSE 12"],["CSE 101"]]}
}

# must pick 1 calc 1 class
calc_1 = {
    "MATH 19A": {"credits": 5, "prerequisites": []},
    "MATH 20A": {"credits": 5, "prerequisites": []}
}

# must pick 1 calc 2 class
calc_2 = {
    "MATH 19B": {"credits": 5, "prerequisites": [["MATH 19A", "MATH 20A"]]},
    "MATH 20B": {"credits": 5, "prerequisites": [["MATH 20A"]]}
}

# must pick 1 calc 3 class
calc_3 = {
    "AM 30": {"credits": 5, "prerequisites": [["AM 10", "MATH 21"], ["MATH 19B", "MATH 20B"]]},
    "MATH 23A": {"credits": 5, "prerequisites": [["MATH 19A"]]}
}

# must pick 1 one linear alg class
linear_alg = {
    "AM 10": {"credits": 5, "prerequisites": []},
    "MATH 21": {"credits": 5, "prerequisites": [["MATH 19A"]]}
}

# must pick 1 stats class
stats = {
    "STAT 131": {"credits": 5, "prerequisites": [["AM 11B", "ECON 11B", "MATH 11B", "MATH 19B", "MATH 20B"]]},
    "CSE 107": {"credits": 5, "prerequisites": [["CSE 16"], ["AM 30", "MATH 22", "MATH 23A"]]}
}

# must pick 1 theory class
theory = {
    "CSE 102": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 103": {"credits": 5, "prerequisites": [["CSE 101"]]}
}

# must pick 1 programming lang class
pl = {
    "CSE 112": {"credits": 5, "prerequisites": [["CSE 101"]]},
    "CSE 114A": {"credits": 5, "prerequisites": [["CSE 101"]]}
}


# must take 3 elective fillers here, plus a capstone that counts as an elective (listed after)
electives = {
    "elective 1": {"credits": 5, "prerequisites": []},
    "elective 2": {"credits": 5, "prerequisites": []},
    "elective 3": {"credits": 5, "prerequisites": []},
}

capstones = {
    "CSE 110B": {"credits": 5, "prerequisites": [["CSE 110A"]]},
    "CSE 115C": {"credits": 5, "prerequisites": [["CSE 115B"]]},
    "CSE 115D": {"credits": 5, "prerequisites": [["CSE 115A"]]},
    "CSE 121": {"credits": 7, "prerequisites": [["CSE 12", "CSE 100", "CSE 100L"],
            ["CSE 13E", "CSE 13S", "ECE 13", "CSE 15", "CSE 15L"],
            ["ECE 101", "ECE 101L", "PHYS 5C", "PHYS 5N"]
        ]
    },
    "CSE 134": {"credits": 5, "prerequisites": [["CSE 120", "CSE 130"]]},
    "CSE 138": {"credits": 5, "prerequisites": [["CSE 130", "CSE 131"]]},
    "CSE 140": {"credits": 5, "prerequisites": [["CSE 101", ["CSE 40", "STAT 132"]]]},
    "CSE 143": {
        "credits": 5,
        "prerequisites": [
            ["CSE 101"],
            ["CSE 107", "STAT 131"],
            ["CSE 40"]
        ]
    },
    "CSE 144": {
        "credits": 5,
        "prerequisites": [
            ["CSE 40", "STAT 132"],
            ["CSE 101"]
        ]
    },
    "CSE 145": {
        "credits": 5,
        "prerequisites": [
            ["CSE 15", "CSE 15L"],
            ["CSE 30"],
            ["CSE 13S"],
            ["AM 30", "MATH 22", "MATH 23A"],
            ["STAT 5", "CSE 107", "STAT 131"],
            ["AM 10", "MATH 21"],
            ["CSE 16", "ECON 113"]
        ]
    },
    "CSE 156": {
        "credits": 5,
        "prerequisites": [["CSE 150", "CSE 101"]],
        "concurrent": "CSE 156L"
    },
    "CSE 156L": {
        "credits": 2,
        "prerequisites": [["CSE 150", "CSE 101"]],
        "concurrent": "CSE 156"
    },
    "CSE 157": {
        "credits": 7,
        "prerequisites": [["CSE 121", "CSE 150"]]
    },
    "CSE 160": {
        "credits": 7,
        "prerequisites": [["CSE 101", ["MATH 21", "AM 10"]]]
    },
    "CSE 161": {
        "credits": 5,
        "prerequisites": [["CSE 160", "equivalent"]],
        "concurrent": "CSE 161L"
    },
    "CSE 161L": {
        "credits": 2,
        "concurrent": "CSE 161"
    },
    "CSE 162": {
        "credits": 5,
        "prerequisites": [["CSE 160", "equivalent"]],
        "concurrent": "CSE 162L"
    },
    "CSE 162L": {
        "credits": 2,
        "concurrent": "CSE 162"
    },
    "CSE 163": {
        "credits": 5,
        "prerequisites": [["CSE 101"]]
    },
    "CSE 168": {
        "credits": 7,
        "prerequisites": [["CSE 160"]]
    },
    "CSE 181": {
        "credits": 5,
        "prerequisites": [["CSE 180", "CSE 130"]]
    },
    "CSE 183": {
        "credits": 5,
        "prerequisites": [
            ["CSE 15", "CSE 15L"],
            ["CMPM 35"],
            ["CSE 101"]
        ]
    },
    "CSE 184": {
        "credits": 5,
        "prerequisites": [["CSE 101"]]
    },
    "CSE 187": {
        "credits": 5,
        "prerequisites": [["CSE 186"]]
    },
    "CMPM 172": {
        "credits": 7,
        "prerequisites": [["CMPM 171"]]
    },
}

# must pick 1 DC course

DCs = {
    "CSE 115A": {"credits": 5, "prerequisites": [["CSE 101"], ["CSE 130"]]},
    "CSE 185E": {"credits": 5, "prerequisites": [["CSE 30", "CSE 12"]]},
    "CSE 185S": {"credits": 5, "prerequisites": [["CSE 101"], ["CSE 101", "CSE 101M", "CSE 120", "CSE 130"]]},
    "CSE 195": {"credits": 5, "prerequisites": []}
}


# generate the schedule
course_schedule = generate_schedule(must_take)

# print the schedule
for quarter, courses in enumerate(course_schedule, start=1):
    print(f"Quarter {quarter}: {', '.join(courses) if courses else 'No courses available'}")


calcs: [['MATH 19A'], ['MATH 19B'], ['MATH 23A'], [], [], [], [], [], [], ['ELECTIVE'], ['ELECTIVE'], ['ELECTIVE', 'CAPSTONE']]
after adding linear alg: [['MATH 19A'], ['MATH 19B'], ['MATH 23A', 'AM 10'], [], [], [], [], [], [], ['ELECTIVE'], ['ELECTIVE'], ['ELECTIVE', 'CAPSTONE']]

after adding STATS TO COURSES: {'CSE 12': {'credits': 7, 'prerequisites': [['CSE 5J', 'CSE 20', 'CSE 30', 'BME 160']]}, 'CSE 16': {'credits': 5, 'prerequisites': [['MATH 20A', 'MATH 19A', 'MATH 19B', 'MATH 11B', 'AM 11B', 'AM 15B', 'ECON 11B']]}, 'CSE 20': {'credits': 5, 'prerequisites': []}, 'CSE 30': {'credits': 5, 'prerequisites': [['CSE 20', 'BME 160', 'MATH 3', 'MATH 11A', 'MATH 19A', 'AM 3', 'AM 11A']]}, 'CSE 40': {'credits': 5, 'prerequisites': [['MATH 19B', 'MATH 20B'], ['CSE 30']]}, 'CSE 13S': {'credits': 7, 'prerequisites': [['CSE 12', 'BME 160']]}, 'ECE 30': {'credits': 5, 'prerequisites': [['MATH 19B', 'MATH 20B']]}, 'CSE 101': {'credits': 5, 'prerequisites': [['CSE 12', 'BME 160'], ['CSE 13E', 